# Game Optimization Project Notebook

Authored By: Simon Lidwell

----------

This project aims to explore game winning optimization algorithms for the sake of having fun and having a deeper understanding of how to win commonly played games like Tic-Tac-Toe and Connect4.

I will start with Tic-Tac-Toe as it is a great introduction to game strategy due to the limited number of possible outcomes. Then I will move into something more complex (Connect4) and do a similar analysis.

The end of this project will be contain a comprehensive analysis of game optimization theory.

## Optimizing Tic-Tac-Toe

In [1]:
from src.utils import TicTacToe

In [2]:
game1 = TicTacToe()

In [3]:
game1.get_or_create_game().model_dump()

{'id': UUID('ff196d7f-664a-48c8-8a24-19697877242d'),
 'createdat': datetime.datetime(2024, 12, 9, 2, 1, 17, 442061),
 'winner': None,
 'game_type': <GameType.TIC_TAC_TOE: 'TicTacToe'>,
 'moves': []}

In [4]:
game1.get_moves()

[]

In [5]:
game1.make_move(player='X', row=1, col=1)

0  0 | 1 | 2 
   --------- 
1  0 | X | 2 
   --------- 
2  0 | 1 | 2 
AI chooses to place at (0, 0)
0  O | 1 | 2 
   --------- 
1  0 | X | 2 
   --------- 
2  0 | 1 | 2 


'Player X make your move'

In [6]:
game1.make_move(player='X', row=0, col=1)

0  O | X | 2 
   --------- 
1  0 | X | 2 
   --------- 
2  0 | 1 | 2 
AI chooses to place at (2, 1)
0  O | X | 2 
   --------- 
1  0 | X | 2 
   --------- 
2  0 | O | 2 


'Player X make your move'

In [7]:
game1.make_move(player='X', row=1, col=0)

0  O | X | 2 
   --------- 
1  X | X | 2 
   --------- 
2  0 | O | 2 
AI chooses to place at (1, 2)
0  O | X | 2 
   --------- 
1  X | X | O 
   --------- 
2  0 | O | 2 


'Player X make your move'

In [8]:
game1.make_move(player='X', row=2, col=0)

0  O | X | 2 
   --------- 
1  X | X | O 
   --------- 
2  X | O | 2 
AI chooses to place at (0, 2)
0  O | X | O 
   --------- 
1  X | X | O 
   --------- 
2  X | O | 2 


'Player X make your move'

# Game Optimization Data Analysis

So I used the backend to capture all the games and moves data into sqlite. This will allow me to observe trends in the game data.

In [14]:
from src.db.dbinit import engine

import polars as pl
from sqlalchemy import text

In [18]:
db = engine.connect()
result_df = pl.read_database(
    query=text(
        """
        SELECT games.id, 
               games.createdat, 
               games.winner, 
               games.game_type, 
               moves.id [move_id],
               moves.player, 
               moves.row,
               moves.col,
               moves.timestamp
        FROM games
            INNER JOIN moves ON games.id = moves.game_id
        WHERE games.winner IS NOT NULL
        """
    ),
    connection=db,
)

db.close()

result_df

2024-12-08 21:23:25,321 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-12-08 21:23:25,322 INFO sqlalchemy.engine.Engine 
        SELECT games.id, 
               games.createdat, 
               games.winner, 
               games.game_type, 
               moves.id [move_id],
               moves.player, 
               moves.row,
               moves.col,
               moves.timestamp
        FROM games
            INNER JOIN moves ON games.id = moves.game_id
        WHERE games.winner is not null
        
2024-12-08 21:23:25,322 INFO sqlalchemy.engine.Engine [generated in 0.00147s] ()
2024-12-08 21:23:25,325 INFO sqlalchemy.engine.Engine ROLLBACK


id,createdat,winner,game_type,move_id,player,row,col,timestamp
str,str,str,str,str,str,i64,i64,str
"""5c36c497-37b9-4ecf-aca4-965dbc…","""2024-12-08 06:02:32.649749""","""X""","""TicTacToe""","""873b7969-7b87-41f8-ae74-051847…","""X""",1,1,"""2024-12-08 06:02:32.652749"""
"""5c36c497-37b9-4ecf-aca4-965dbc…","""2024-12-08 06:02:32.649749""","""X""","""TicTacToe""","""bb1c7d83-6d40-4189-b2f2-8a18d2…","""O""",0,1,"""2024-12-08 06:02:32.652749"""
"""5c36c497-37b9-4ecf-aca4-965dbc…","""2024-12-08 06:02:32.649749""","""X""","""TicTacToe""","""eb09e284-6376-4485-8826-d7d3f4…","""X""",0,0,"""2024-12-08 06:02:32.652749"""
"""5c36c497-37b9-4ecf-aca4-965dbc…","""2024-12-08 06:02:32.649749""","""X""","""TicTacToe""","""d829562d-9ea6-4b33-abdf-02a89e…","""O""",2,2,"""2024-12-08 06:02:32.652749"""
"""5c36c497-37b9-4ecf-aca4-965dbc…","""2024-12-08 06:02:32.649749""","""X""","""TicTacToe""","""d0d09f9d-2984-4829-952e-b647ad…","""X""",1,2,"""2024-12-08 06:02:32.652749"""
…,…,…,…,…,…,…,…,…
"""ff196d7f-664a-48c8-8a24-196978…","""2024-12-09 02:01:17.442061""","""Tie""","""TicTacToe""","""25d09eac-9e1d-498e-8978-f846ea…","""X""",1,0,"""2024-12-09 02:01:17.444062"""
"""ff196d7f-664a-48c8-8a24-196978…","""2024-12-09 02:01:17.442061""","""Tie""","""TicTacToe""","""fedc7d55-5df7-45ae-8bef-0ceb89…","""O""",1,2,"""2024-12-09 02:01:17.444062"""
"""ff196d7f-664a-48c8-8a24-196978…","""2024-12-09 02:01:17.442061""","""Tie""","""TicTacToe""","""a85487fb-365a-4e08-8ba8-42d3f4…","""X""",2,0,"""2024-12-09 02:01:17.444062"""
